In [2]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2
import pandas as pd
from src.data import RAW_DATA_PATH
from src.data.dataset import load_raw_data

In [3]:
df = load_raw_data(RAW_DATA_PATH)

In [4]:
df.head()

,seq_guid,timestamp,value,date
0,54ad727ddc76040900d55051|82267c03919ef18dabc33...,2017-09-27T11:00:00.000Z,2754,2017-09-27
1,54b80cb0cb084b0c00294f2a|6ce908fc9895fdfc035e4...,2018-06-08T01:00:00.000Z,538,2018-06-08
2,54bde3073deefe0c00a01401|46e37148a68739387c48b...,2017-11-28T02:00:00.000Z,1614,2017-11-28
3,54c6ea8213078c0c001cec71|044a408e374c8a102b709...,2018-04-02T03:00:00.000Z,22105,2018-04-02
4,54e643cd95962f0c004cde71|ee74f9800708efd6b39c3...,2018-05-29T00:00:00.000Z,7803,2018-05-29


In [65]:
# null value check
df.isnull().any()

seq_guid     False
timestamp    False
value        False
dtype: bool

In [66]:
f"{len(df.seq_guid.unique())} unique seq_guids."

'51534 unique seq_guids.'

In [67]:
f"Number of (seq_guid, timestamp) collisions = \
{len(df[df.duplicated(subset=['seq_guid', 'timestamp'], keep=False)])}"

'Number of (seq_guid, timestamp) collisions = 0'

In [68]:
# convert timestamp to datetime object and rewindow to one day
df['date'] = (
    pd.to_datetime(df.timestamp, format='%Y-%m-%dT%H:%M:%S.%fZ')
    .dt.normalize()
)

In [69]:
f"Number of (seq_guid, date) collisions = \
{len(df[df.duplicated(subset=['seq_guid', 'date'], keep=False)])}"

'Number of (seq_guid, date) collisions = 974'

In [70]:
f"Number of (seq_guid, date, value) collisions; i.e. duplicate rows after re-windowing = \
{len(df[df.duplicated(subset=['seq_guid', 'date', 'value'], keep=False)])}"

'Number of (seq_guid, date, value) collisions; i.e. duplicate rows after re-windowing = 970'

In [72]:
idx = (
    df.duplicated(subset=['seq_guid', 'date'], keep=False) 
    ^ df.duplicated(subset=['seq_guid', 'date', 'value'], keep=False)
)
ambiguous_records = df[idx].sort_values('date')
print(f"There are {len(ambiguous_records)} rows with ambiguous dates.")
ambiguous_records

There are 4 rows with ambiguous dates.


,seq_guid,timestamp,value,date
34266,567b1c2b25684d18004c8e8e|d91ff4b3385e2980a277b...,2017-10-30T07:00:00.000Z,3390,2017-10-30
337533,567b1c2b25684d18004c8e8e|d91ff4b3385e2980a277b...,2017-10-30T00:00:00.000Z,6780,2017-10-30
82758,54e6b8ebca8f520c001bbfd4|5b0a591284c00a911fbbf...,2017-11-27T00:00:00.000Z,1000,2017-11-27
193283,54e6b8ebca8f520c001bbfd4|5b0a591284c00a911fbbf...,2017-11-27T08:00:00.000Z,500,2017-11-27


In [5]:
df.value.min()

1